In [ ]:
"""Урок 1. Отношения между таблицами в sqlalchemy: 1-2-1, 1-2-many"""

In [ ]:
"""
Давайте вспомним про внешние ключи. Внешние ключи позволяют установить связи между таблицами. Они устанавливается для столбцов зависимой таблицы, и указывают на один из столбцов главной таблицы. Чаще всего, внешний ключ указывает на первичный ключ из связанной главной таблицы. Таким образом, создается ссылочная целостность между таблицами. Внешний ключ, аналогично первичному, может состоять из нескольких столбцов и называться составным. В таком случае, почти всегда, составной внешний ключ ссылается на таблицу с составным первичным ключом.

В ORM SQLAlchemy есть два варианта создания внешних ключей. Первый и наиболее популярный вариант - это определить объект ForeignKey в качестве аргумента объекта Column. Второй вариант это использовать конструкцию ForeignKeyConstraint на уровне таблицы. Давайте подробнее поговорим про каждый из них.

   parent_id = Column(Integer, ForeignKey("parent.id"), nullable=False), 
 где “parent.id” - <Имя таблицы> . <Имя столбца>

Объект ForeignKey определяет зависимость между двумя столбцами. Обязательным аргументом внешнего ключа, чаще всего, является строка вида имя таблицы точка имя столбца. В случае, если вы используете схему, перед названием таблице еще нужно указать имя схемы. Преимущество использования строки заключается в том, что связь между таблицами определяется только один раз при создании таблиц поэтому мы легко можем разбить наши модели по разным модулям и определять их в любом порядке.

Обратите внимание, что при использовании объекта ForeignKey создать составной внешний ключ не получится. Точнее несколько внешний ключей описать получиться, но SQLALchemy не поймет, что оба ключи объединены вместе. Для корректного определения и нужен второй вариант - использование конструкции ForeignKeyConstraint. 

   ForeignKeyConstraint(columns=[“parent_id”, “parent_second_key’'], ref_columns=[“parent.id”, “parent.second_key”])

Данная конструкция определяет внешние ключи на уровне таблицы, то есть задавать ключи в таком случае мы будем не внутри объекта Column, а при описании объектов таблицы. 

Обязательными параметрами конструкцию являются columns - список колонок внешних ключей и ref_columns - список колонок, на который эти внешние ключи будут ссылаться.

Стоит отметить, что для простого внешнего ключа объект ForeignKey, который определяется внутри колонки, является эквивалентом конструкции ForeignKeyConstraint, определяемой на уровне таблицы.

Зная, как связывать таблицы между собой, давайте поговорим об основных моделях взаимоотношений между таблицами.
"""

In [ ]:
"""
Первая связь, которую мы разберем это связь один ко многим.
В данной типе связей одной записи родительской таблицы соответствует множество записей в дочерней таблице. У родителя несколько детей, у учителя множество учеников, одну и ту же должность занимает множество сотрудников. 
"""
from sqlalchemy import Column, Integer, ForeignKey, create_engine
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///one_to_many.db')
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

class Parent(Base):
   __tablename__ = 'parent'
   id = Column(Integer, primary_key=True)
   children = relationship("Child", back_populates="parent")

   # альтернативый вариант двунаправленной связи
   # children = relationship("Child", backref="parent")


class Child(Base):
   __tablename__ = 'child'
   id = Column(Integer, primary_key=True)
   parent_id = Column(Integer, ForeignKey('parent.id'))
   parent = relationship("Parent", back_populates="children")


# связываем родительскую таблицу в класическом стиле представления
# mapper(Parent, properties={
#     'children': relationship(Child)
# })

if __name__ == '__main__':

   Base.metadata.create_all(engine)

   parent = Parent()
   session.add(parent)
   session.commit()
   child_one = Child(parent_id=1)
   child_two = Child(parent_id=1)
   session.add(child_one)
   session.add(child_two)
   session.commit()

   my_children = session.query(Child).filter(Child.parent_id==1).all()
   my_parent = session.query(Parent).first()



In [ ]:
"""
Связь определяется внешним ключом внутри дочерней таблицы, который ссылается на колонку таблицы-родителя. В родительской таблице связь описывается с помощью функции relationship(), как ссылка на список дочерних элементов. Эта функция обеспечивает связь между двумя классами ORM. 
Обязательным аргументов функции relationship является дочерняя таблица. 
В случае использование классического стиля работы с ORM, нужно вызвать mapper для родителя и определить функцию отношения. Mapper необходимо вызывать после описания и родительской и дочерний таблиц, иначе ORM не сможет связать таблицы.

Данная проблема отсутствует при использовании декларативного представления. В декларативном стиле мы можем передавать объект дочерней таблицы, как строковый аргумент. Эти строковые аргументы преобразуются в объекты, которые оценивают строку как код Python, используя список ORM - классов в качестве пространства имен. Это позволяет выполнять поиск дочернего класса по строковому имени и устраняет необходимость импорта до объявления родительского класса.
Чтобы установить двунаправленную связь в режиме один ко многим, где со стороны родителя -  много к одному, укажите дополнительную relationship() у дочернего класса и соедините их с помощью параметра back_populates. Альтернативный, и как по мне, более удобный вариант, использоваться параметр backref у родителя. Он автоматически создаст двунаправленную связь и нет необходимости определять relationship у ребенка, она будет создана автоматически.
В целом, как отдельный тип связи его не выделяют, но давайте посмотрим, как реализуется связь много-к-одному. Если при связи один-ко-многим у родителя могут быть несколько детей, то здесь наоборот, у ребенка множество родителей, папа и мама.
"""

from sqlalchemy import Column, Integer, ForeignKey, create_engine
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
 
engine = create_engine('sqlite:///many_to_one.db')
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()
 
 
class Parent(Base):
   __tablename__ = 'parent'
   id = Column(Integer, primary_key=True)
   child_id = Column(Integer, ForeignKey('child.id'))
   child = relationship("Child")
   # двунаправленная связь
   # child = relationship("Child", back_populates="parents")
   # альтернативый вариант двунаправленной связи
   # child = relationship("Child", backref="parents")
 
 
class Child(Base):
   __tablename__ = 'child'
   id = Column(Integer, primary_key=True)
   # двунаправленная связь
   # parents = relationship("Parent", back_populates="child")
 
 
if __name__ == '__main__':
   Base.metadata.create_all(engine)
 
   child = Child()
   session.add(child)
   session.commit()
   mother = Parent(child_id=1)
   father = Parent(child_id=1)
   session.add(mother)
   session.add(father)
   session.commit()
 
   parents = session.query(Parent).filter(Parent.child_id == 1).all()
   child = session.query(Child).first()
"""
Здесь мы поместили внешний ключ в родительскую таблицу, ссылаясь на дочернюю, связь relationship так же определяем в родительской таблице.
"""


In [ ]:
"""
Следующий тип связи это один к одному. Данный тип встречается нечасто. Например, есть необходимость, в целях безопасности, разбить хранение сущности на несколько таблиц, конфиденциальные данные сотрудника в одной таблице, приватные в другой. 
"""
from sqlalchemy import Column, Integer, ForeignKey, create_engine
from sqlalchemy.orm import relationship, sessionmaker, backref
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///one_to_one.db')
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()


class Parent(Base):
   __tablename__ = 'parent'
   id = Column(Integer, primary_key=True)
   # children = relationship("Child", back_populates="parent")

   # преобразуем в скалярную связь один к одному
   child = relationship("Child", back_populates="parent", uselist=False)


class Child(Base):
   __tablename__ = 'child'
   id = Column(Integer, primary_key=True)
   parent_id = Column(Integer, ForeignKey('parent.id'), unique=True)
   # parent = relationship("Parent", back_populates="children")

   # переименуем back_populates
   parent = relationship("Parent", back_populates="child")

   # определение с помощью backref
   # parent = relationship("Parent",
   #                       backref=backref("child", uselist=False))


if __name__ == '__main__':
   Base.metadata.create_all(bind=engine)
   parent = Parent()
   session.add(parent)
   session.commit()
   child_one = Child(parent_id=1)
   session.add(child_one)
   session.commit()

   # child_two = Child(parent_id=1)
   # session.add(child_two)
   # session.commit()

   # проверить родителя
   check_parent = session.query(Parent).filter_by(id=1).one()
   print('check')

   # проверим детей (дети привязываются к родителю все равно)
   children = session.query(Child).all()
   print('check')



"""
Один к одному - это, по сути, двунаправленная связь, которую мы разбирали в случае один ко многим. Внутри ORM принцип «один-к-одному» рассматривается как соглашение, согласно которому подразумевается, что для любой родительской строки будет существовать только одна дочерняя строка. Достигается это соглашение с помощью параметра use_list внутри функции relationship. Этот булевый параметр определяет, нужно ли подгружать атрибут в виде списка или единственного значения. Соответственно, нас интересует use_list = False, чтобы запретить связывать множество объектов с одним.
Если мы определим несколько детей для родителя, то за счет атрибута use_list, привяжется только первый объект. Однако, есть одно но. В ORM не предусмотрен алгоритм предотвращения создания нескольких объектов с привязкой к одному родителю. То есть, в таблице детей у нас может быть определены строки с привязкой к одному родителю. Для того, чтобы решить данную проблему, нам самостоятельно необходимо определить уникальность внешнего ключа. Это сделать мы можем с помощью конструкции UniqueConstraint на уровне таблицы, либо с помощью параметра unique при описании колонки.
Стоит отметить, что соглашение один-к-одному можно также определить с помощью функции backref.
"""


In [ ]:
"""Последняя связь, которую мы разберем - связь много-ко-многим.
У родителей есть два ребенка. У папы и мамы есть сын и дочь, соответственно у дочери и сына есть и папа и мама. То есть несколько строк одной таблицы соответствуют нескольким строкам другой таблицы.
Для реализации связи, нам необходимо создать интеграционную таблицу, в который будем хранить связи между двумя таблицами.
"""

from sqlalchemy import Column, Integer, ForeignKey, create_engine, Table
from sqlalchemy.orm import relationship, sessionmaker, backref
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///many_to_many.db')
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

integration_table = Table('integrations', Base.metadata,
                         Column('parent_id', ForeignKey('parent.id'),
                                primary_key=True),
                         Column('child_id', ForeignKey('child.id'),
                                primary_key=True)
                         )


class Parent(Base):
   __tablename__ = 'parent'
   id = Column(Integer, primary_key=True)
   # children = relationship("Child",
   #                         secondary=integration_table)

   # двунаправленная связь
   children = relationship(
       "Child",
       secondary=integration_table,
       back_populates="parents")

   # определение с помощью backref
   # children = relationship("Child",
   #                         secondary=integration_table,
   #                         backref="parents")


class Child(Base):
   __tablename__ = 'child'
   id = Column(Integer, primary_key=True)

   # двунаправленная связь
   parents = relationship(
       "Parent",
       secondary=integration_table,
       back_populates="children")


if __name__ == '__main__':
   Base.metadata.create_all(bind=engine)

   father = Parent()
   mother = Parent()

   son = Child()
   daughter = Child()

   # добавим отцу детей
   # children - коллекция детей
   father.children.extend([son, daughter])


   # обратная ситуация - добавим сын и дочери маму (при двунаправленной связи)
   son.parents.append(mother)
   daughter.parents.append(mother)

   session.add(father)
   session.add(mother)
   session.add(son)
   session.add(daughter)
   session.commit()

   my_parents = session.query(Parent).all()
   my_children = session.query(Child).all()

   many_to_many_data = session.query(integration_table).all()

   father.children.remove(daughter)
   # как себя поведет интеграционная таблица?
   # session.delete(son)

"""
Создадим объект типа Table для интеграционной таблицы. Это классический стиль представления таблиц, значит мы должны передать в объект название таблицы, метадату и список колонок. Метадату берем из Base.metadata. Для того, чтобы связать интеграционную таблицу с основной мы должны использовать параметр secondary функции relationship(). В secondary можем передать как сам объект таблицы, так и строковое имя этой таблицы, в случае, если мы работаем в декларативном стиле. Это будет означать, что список зависимых элементов дочерней таблицы хранится в интеграционной таблице. В integration_table я определил составной первичный ключ из обоих внешних ключей, для того чтобы соблюдать уникальность связей. Также, данную связь можно сделать двунаправленной и описать с помощью backref().

"""